In [1]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

#This line will hide code by default when the notebook is exported as HTML
#di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import hail as hl
hl.init(tmp_dir='/net/scratch/people/plggosborcz')

Running on Apache Spark version 2.4.3
SparkUI available at http://p0637.prometheus:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.62-84fa81b9ea3d
LOGGING: writing to /net/archive/groups/plggneuromol/sportsmen-wgs/analysis/imdik-zekanowski-sportwgs/hail-20210529-1945-0.2.62-84fa81b9ea3d.log


In [3]:
from hail.plot import show
from pprint import pprint
from bokeh.layouts import gridplot
hl.plot.output_notebook()


import numpy as np
import pandas as pd
from functools import reduce
from itertools import chain

from bokeh.plotting import output_notebook, show, figure
from bokeh.palettes import viridis

output_notebook()

Loading BokehJS ...

Loading BokehJS ...

In [12]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/joint-unrelated-corrected.mt')

In [13]:
sportsmen=['B102','B156','B24','B338','B382','B399','B427','B430','B431','B432','B433','B435','B436','B437','B438','B439','B441','B442','B443','B444','B445','B446','B448','B449','B450','B451','B452','B453','B454',
'B455','B456','B457','B458','B459','B461','B462','B463','B464','B465','B466','B467','B468','B469','B470','B471','B472','B473','B474','B475','B476','B477','B478','B479','B480','B481','B482','B483','B485','B486','B487','B488',
'B489','B490','B491','B492','B493','B494','B495','B496','B497','B498','B499','B501','B502','B503','B504','B505','B506','B507','B508','B509','B515','B518','B522','B523','B524','B525','B526','B527','B528','B529','B530','B531',
'B532','B533','B534','B535','B536','B537','B538','B539','B81']

In [14]:
len(sportsmen)

102

In [15]:
mt.count()

(9824949, 147)

In [16]:
mt = mt.filter_cols(hl.set(sportsmen).contains(mt.s))

In [17]:
genes = hl.read_table('/net/archive/groups/plggneuromol/GTS-analysis/analysis/vcf_preprocessing/genecode_v32.ht')

genes = genes.filter(hl.is_valid_contig(genes['hg38.knownGene.chrom'], reference_genome='GRCh38'))

hpo = hl.import_table('/net/archive/groups/plggneuromol/GTS-analysis/analysis/vcf_preprocessing/hpo.tsv', impute = True, no_header=True)

start = genes['hg38.knownGene.txStart']
stop =  genes['hg38.knownGene.txEnd']

genes = genes.transmute(interval = 
                        hl.locus_interval(genes['hg38.knownGene.chrom'], 
                                          start,
                                          stop,
                                          reference_genome='GRCh38', includes_start=False))

genes = genes.key_by(genes['hg38.kgXref.geneSymbol'])

hpo = hpo.key_by(hpo.f0)

genes = genes.annotate(hpo = hpo.index(genes['hg38.kgXref.geneSymbol'], all_matches = True)['f1'])

genes = genes.key_by(genes.interval)

mt = mt.annotate_rows(within_gene = hl.array(hl.set(genes.index(mt.locus, all_matches=True)['hg38.kgXref.geneSymbol'])))
mt = mt.annotate_rows(hpo = hl.array(hl.set(genes.index(mt.locus, all_matches=True)['hpo'])))
  

2021-05-29 18:39:10 Hail: INFO: Reading table to impute column types
2021-05-29 18:39:12 Hail: INFO: Finished type imputation
  Loading field 'f0' as type str (imputed)
  Loading field 'f1' as type str (imputed)


In [18]:
mt.write('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/sportsmen-only.mt')

2021-05-29 18:39:16 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-05-29 18:39:17 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-05-29 18:39:18 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-05-29 18:39:19 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-05-29 18:39:19 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-05-29 18:39:20 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-05-29 18:41:05 Hail: INFO: wrote matrix table with 9824949 rows and 100 columns in 3061 partitions to /net/archive/groups/plggneuromol/sportsmen-wgs/mts/sportsmen-only.mt
    Total size: 5.60 GiB
    * Rows/entries: 5.60 GiB
    * Columns: 419.00 B
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  7054 rows (7.38 MiB)


In [19]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/sportsmen-only.mt')

In [25]:
#now filter with the mitocarta genes

mito = hl.import_table('/net/archive/groups/plggneuromol/sportsmen-wgs/analysis/gene-lists-and-results/mitocarta-genes.csv', impute = True)

2021-05-29 18:49:46 Hail: INFO: Reading table to impute column types
2021-05-29 18:49:46 Hail: INFO: Finished type imputation
  Loading field 'Symbol' as type str (imputed)


In [28]:
mt = mt.filter_rows(hl.any(lambda x: hl.literal(mito.Symbol.collect()).contains(x), mt.within_gene))

In [29]:
mt.write('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/mito-only.mt')

2021-05-29 18:51:43 Hail: INFO: wrote matrix table with 172179 rows and 100 columns in 3061 partitions to /net/archive/groups/plggneuromol/sportsmen-wgs/mts/mito-only.mt
    Total size: 114.82 MiB
    * Rows/entries: 114.82 MiB
    * Columns: 419.00 B
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  3952 rows (2.95 MiB)


In [30]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/mito-only.mt')

## lets further filter variants - higher quality qenotypes only

In [31]:
#for some reason a loop doesn't work here
rpmk = hl.read_table('/net/archive/groups/plggneuromol/sportsmen-wgs/analysis/vcf_preprocessing/repeatmasker_anno.ht')
temp = rpmk.key_by(rpmk['larger_int_start'])
mt = mt.filter_rows(hl.is_defined(temp[mt.locus]), keep = False)
mt.checkpoint('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/mito-filtered0.mt')
    
rpmk = hl.read_table('/net/archive/groups/plggneuromol/sportsmen-wgs/analysis/vcf_preprocessing/repeatmasker_anno.ht')
temp = rpmk.key_by(rpmk['larger_int_end'])
mt = mt.filter_rows(hl.is_defined(temp[mt.locus]), keep = False)
mt.checkpoint('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/mito-filtered1.mt')

rpmk = hl.read_table('/net/archive/groups/plggneuromol/sportsmen-wgs/analysis/vcf_preprocessing/repeatmasker_anno.ht')
temp = rpmk.key_by(rpmk['larger_int_start_2'])
mt = mt.filter_rows(hl.is_defined(temp[mt.locus]), keep = False)
mt.checkpoint('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/mito-filtered2.mt')


2021-05-29 19:02:24 Hail: INFO: Coerced sorted dataset
2021-05-29 19:02:50 Hail: INFO: wrote matrix table with 172179 rows and 100 columns in 3061 partitions to /net/archive/groups/plggneuromol/sportsmen-wgs/mts/mito-filtered0.mt
    Total size: 146.50 MiB
    * Rows/entries: 146.50 MiB
    * Columns: 432.00 B
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  3952 rows (3.67 MiB)
2021-05-29 19:02:52 Hail: INFO: Coerced sorted dataset
2021-05-29 19:02:55 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-05-29 19:04:11 Hail: INFO: wrote matrix table with 171323 rows and 100 columns in 3061 partitions to /net/archive/groups/plggneuromol/sportsmen-wgs/mts/mito-filtered1.mt
    Total size: 145.84 MiB
    * Rows/entries: 145.84 MiB
    * Columns: 432.00 B
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  3935 rows (3.66 MiB)
2021-05-29 19:04:14 Hail: INFO: Coerced sorted dataset
2021-05-29 19:04

In [32]:
rpmk = hl.read_table('/net/archive/groups/plggneuromol/sportsmen-wgs/analysis/vcf_preprocessing/repeatmasker_anno.ht')
temp = rpmk.key_by(rpmk['larger_int_end_2'])
mt = mt.filter_rows(hl.is_defined(temp[mt.locus]), keep = False)
mt.checkpoint('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/mito-filtered3.mt')  

2021-05-29 19:05:46 Hail: INFO: Coerced sorted dataset
2021-05-29 19:05:48 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-05-29 19:05:50 Hail: INFO: Coerced sorted dataset
2021-05-29 19:05:53 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-05-29 19:11:12 Hail: INFO: wrote matrix table with 169531 rows and 100 columns in 3061 partitions to /net/archive/groups/plggneuromol/sportsmen-wgs/mts/mito-filtered3.mt
    Total size: 144.42 MiB
    * Rows/entries: 144.42 MiB
    * Columns: 432.00 B
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  3893 rows (3.62 MiB)


In [23]:
mt = hl.read_matrix_table('/net/archive/groups/plggneuromol/sportsmen-wgs/mts/mito-filtered3.mt')

In [24]:
mt = hl.variant_qc(mt)

In [25]:
mt = mt.annotate_rows(info = hl.struct(AC = mt.AC, 
                                       AF = mt.AF,
                                       AN = mt.AN,
                                       rsid = mt.rsid,
                                       gnomAD_v3_1_allele_frequency = mt.gnomad_v3.v3_nfe.AF,
                                       gnomAD_v3_1_homozygote_count = mt.gnomad_v3.v3_nfe.homozygote_count,
                                       predicted_most_severe_consequence = mt.gnomad_v3.most_severe_consequence,
                                       transcript_id = mt.gnomad_v3.transcript_consequences.transcript_id, 
                                       pholyphen_prediction = mt.gnomad_v3.transcript_consequences.polyphen_prediction, 
                                       polyphen_score = mt.gnomad_v3.transcript_consequences.polyphen_score, 
                                       sift_prediction = mt.gnomad_v3.transcript_consequences.sift_prediction, 
                                       sift_score = mt.gnomad_v3.transcript_consequences.sift_score,
                                       gene = mt.within_gene,
                                       hpo = hl.flatten(mt.hpo),
                                       mean_dp = mt.variant_qc.dp_stats.mean,
                                       mean_gq = mt.variant_qc.gq_stats.mean,
                                       call_rate = mt.variant_qc.call_rate
                                      ))

In [26]:
mt = mt.select_rows(mt.info)

In [27]:
mt.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'info': struct {
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        rsid: str, 
        gnomAD_v3_1_allele_frequency: float64, 
        gnomAD_v3_1_homozygote_count: int32, 
        predicted_most_severe_consequence: str, 
        transcript_id: array<str>, 
        pholyphen_prediction: array<str>, 
        polyphen_score: array<float64>, 
        sift_prediction: array<str>, 
        sift_score: array<float64>, 
        gene: array<str>, 
        hpo: array<str>, 
        mean_dp: float64, 
        mean_gq: float64, 
        call_rate: float64
    }
----------------------------------------
Entry fields:
    'AD': array<int32>
    'DP': int32
    'GQ': int32
    'GT': call
    'MIN_DP': int32
    'PGT': call
    

In [28]:
hl.export_vcf(mt,'/net/archive/groups/plggneuromol/sportsmen-wgs/mts/mito.vcf.bgz')

2021-05-29 20:08:01 Hail: INFO: merging 3061 files totalling 101.4M...
2021-05-29 20:08:07 Hail: INFO: while writing:
    /net/archive/groups/plggneuromol/sportsmen-wgs/mts/mito.vcf.bgz
  merge time: 5.817s
